# 🤖 Applying State-of-the-Art NLP

## 📡 Remote Models

The big LLM companies make their models available via APIs. This means that you can use them without having to run the model on your own hardware. At a basic level, it can be very similar to using the web interface of tools like ChatGPT, but you can also use them in a much more customised way in your own code to build applications.

We will experiment with the OpenAI API, but the same principles (and often code syntax) apply to other APIs such as Anthropic, Google, and DeepSeek. OpenAI provides plenty of [documentation](https://platform.openai.com/docs/overview) and [examples](https://cookbook.openai.com/) in a 'cookbook' format.

These APIs do cost money, but the smaller models are particularly affordable (several pence per million tokens in / out) and the larger models are often still cheaper than running your own hardware. Today we will be using a shared account (please do not abuse it), I will share with you the API key.

In [ ]:
from openai import OpenAI

# I am providing the API key in plain text here. However, you shoulld never hardcode your API key in the code, and certainly not upload it to GitHub
# Look into environment variables (the dotenv package is useful for this) to protect your accounts

API_KEY = ""

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=API_KEY)

# Example usage of the OpenAI client to generate a response
response = client.responses.create(
    model="gpt-4.1", # Specify the model you want to use, certain models are more powerful / expensive than others
    input="Write a one-sentence bedtime story about a the London School of Economics and Political Science."
)

print(response.output_text)

We can also see a bunch of metadata associated with the response

In [ ]:
print(response)

## 🏷️ LLMs as classifiers / labellers

Often in social science research, we want to label or classify text data. Traditionally, this would be done using using human annotation, or simple keyword matching, with more recent efforts using machine learning models trained on human-labelled data.

We can leverage the advanced capabilities of LLMs to perform this task without needing to train a model ourselves. We will focus on a simple binary classifcation task here, but the same principles can be applied to more complex tasks.

Three different approaches to labelling text data with LLMs:
1. **Zero-shot classification**: We can use the LLM to classify text without any training data. We simply provide the text and the labels we want to classify it into, and the LLM will return a label for each piece of text.
2. **Few-shot classification**: We can provide the LLM with a few examples of text and their corresponding labels, and the LLM will use this information to classify new text. This is similar to how we would train a machine learning model, but we don't need to do any formal training ourselves.
3. **Fine-tuning**: We can 'fine-tune' a pre-trained LLM on our own data, which allows us to create a custom model that is tailored to our specific task. This is more complex and requires more resources, but can lead to better performance. We won't do this today, but it is possible to do locally and using the OpenAI API.

We will focus on performing a binary text classification task. We will take Opinion pieces from the Guardian news dataset and classify whether they are supportive of the UK's approach to managing the COVID-19 pandemic or not.

In [ ]:
import pandas as pd

labeled_df = pd.read_csv('data/covid_stories_labeled.csv', index_col=0) # Contains articles labelled by human annotators as supportive or not of the UK's approach
unlabeled_df = pd.read_csv('data/covid_stories_unlabeled.csv', index_col=0) # Contains articles that have not been labelled yet

display(labeled_df)
display(unlabeled_df)

In [ ]:
labeled_df['label'].value_counts()

We see from the labelled sample that this is quite imbalanced data, with only a small number of articles being supportive of the UK's approach to managing the pandemic.

### 0️⃣ Zero-shot classification


We provide a 'system prompt' for the LLM to follow, we then provide the text that we want to classify. Since we are interested in it returning strictly structured output, we will use a JSON schema to specify the output we want. With our example here it's quite simple, but more complex schemas can be used to return more complex data structures.

In [ ]:
system_prompt = "You are a classification model that predicts whether a news article is supportive of the UK's approach to managing the COVID-19 pandemic (1) or not (0)."
article_text = labeled_df['bodyContent'].iloc[0] # Example article text from the dataset

response = client.responses.create(
  model="gpt-4o-mini", # We select the smaller, cheaper model for this task
  input=[
    {
      # This part defines the system prompt, which sets the context for the model
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": system_prompt
        }
      ]
    },
    {
      # This part defines the user input, which is the article text to be classified
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": article_text
        }
      ]
    },
  ],

  # This next part defines the output format, a JSON object with a single key "classification"
  text={"format": {
        "name": "news_article_classification",
        "type": "json_schema",
        "schema": {
            "type": "object",
            "properties": {
            "classification": {
                "type": "number",
                "description": "The classification output of the model: 1 for supportive of UK's approach to COVID, 0 otherwise."
            }
            },
            "required": [
            "classification"
            ],
            "additionalProperties": False
        },
        "strict": True
        }
    },
)

In [ ]:
import json

#  The output is returned as a string, which wan quickly covert to proper JSON / dict format
print(response.output_text)
print(type(response.output_text))

formatted_output = json.loads(response.output_text)
print(formatted_output)
print(type(formatted_output))

Let's package this up into a function and apply it to the full training dataframe.

In [ ]:
import json

def zeroshot_classify(article_text):
    response = client.responses.create(
        model="gpt-4o-mini", # We use the smaller, cheaper model for this task
        input=[
            {
                # This part defines the system prompt, which sets the context for the model
                "role": "system",
                "content": [
                    {
                        "type": "input_text",
                        "text": system_prompt
                    }
                ]
            },
            {
                # This part defines the user input, which is the article text to be classified
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": article_text
                    }
                ]
            },
        ],

        # This next part defines the output format, a JSON object with a single key "classification"
        text={"format": {
            "name": "news_article_classification",
            "type": "json_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "classification": {
                        "type": "number",
                        "description": "The classification output of the model: 1 for supportive of UK's approach to COVID, 0 otherwise."
                    }
                },
                "required": ["classification"],
                "additionalProperties": False
            },
            "strict": True
        }},
    )
    return json.loads(response.output_text)['classification']

In [ ]:
# Apply to all articles in the labeled dataset
labeled_df['predicted_zero-shot'] = labeled_df['bodyContent'].apply(zeroshot_classify)
display(labeled_df['predicted_zero-shot'])

### 🔢 Few-shot classification
In contrast to zero-shot classification, we can provide the LLM with a few examples of text and their corresponding labels. This in principle allows the LLM to learn from these examples and apply that knowledge to classify new text. However, it may overfit to the examples we provide, so we need to be careful about the examples we choose.

In [ ]:
# Get some positive and negative examples for few-shot learning
few_shot_examples_df = pd.read_csv('data/covid_stories_fewshot_examples.csv', index_col=0)  # Contains a few-shot dataset with examples of both classes

# Format the few-shot examples for the OpenAI API, we treat each example as a pair of user and assistant messages
few_shot_examples = [
    ({
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": x['bodyContent']
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "output_text",
                "text": f'{{"classification":{x['label']}}}' # replace with the following if you get a bug: '{"classification":{%s}}' %x['label']
            }
        ]
    })
for _, x in few_shot_examples_df.iterrows()]

few_shot_examples = [y for x in few_shot_examples for y in x]

few_shot_examples

In [ ]:
# Few-shot classification using OpenAI API

def fewshot_classify(article_text, few_shot_examples):
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": system_prompt
                }
            ]
        }
    ] + few_shot_examples + [ # Add few-shot examples to the messages
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": article_text
                }
            ]
        }
    ]
    response = client.responses.create(
        model="gpt-4o-mini",
        input=messages,
        text={"format": {
            "name": "news_article_classification",
            "type": "json_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "classification": {
                        "type": "number",
                        "description": "The classification output of the model: 1 for supportive of UK's approach to COVID, 0 otherwise."
                    }
                },
                "required": ["classification"],
                "additionalProperties": False
            },
            "strict": True
        }},
    )
    return json.loads(response.output_text)['classification']


In [ ]:
# Test on one article
result = fewshot_classify(article_text, few_shot_examples)
print(result)

In [ ]:
# Apply to all articles in the labeled dataset
labeled_df['predicted_few-shot'] = labeled_df['bodyContent'].apply(fewshot_classify, few_shot_examples=few_shot_examples)

## ✅ VALIDATION

I mentioned this at the start of today. **We should always validate our results**, rather than blindly trusting the models, no matter how advanced they are.

We will compare the zero-shot and few-shot classification results against our "ground truth" human labels. We will use the `sklearn` library to calculate the accuracy, precision, recall, and F1 score of our models.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Generate classification report for zero-shot predictions
print('Accuracy (zero-shot):', accuracy_score(labeled_df['label'], labeled_df['predicted_zero-shot']))
print('Precision (zero-shot):', precision_score(labeled_df['label'], labeled_df['predicted_zero-shot']))
print('Recall (zero-shot):', recall_score(labeled_df['label'], labeled_df['predicted_zero-shot']))
print('F1 Score (zero-shot):', f1_score(labeled_df['label'], labeled_df['predicted_zero-shot']))

# Generate classification report for few-shot predictions
print('Accuracy (few-shot):', accuracy_score(labeled_df['label'], labeled_df['predicted_few-shot']))
print('Precision (few-shot):', precision_score(labeled_df['label'], labeled_df['predicted_few-shot']))
print('Recall (few-shot):', recall_score(labeled_df['label'], labeled_df['predicted_few-shot']))
print('F1 Score (few-shot):', f1_score(labeled_df['label'], labeled_df['predicted_few-shot']))


We choose the approach with the best performance (if satisfactory), and then apply it to the rest of our dataset, rather than spending lots of human time labelling the data ourselves. It looks like the zero-shot classification is performing well enough, so we will use that to label the rest of our dataset.

In [ ]:
unlabeled_df['predicted'] = unlabeled_df['bodyContent'].apply(zeroshot_classify)

### 🏋️ Exercise
1. We used the simple, fast, cheap `gpt-4o-mini` model. Try using the more powerful `gpt-4o` model instead on our labeled data. How does it perform? Can you think of a task (perhaps from your own work) where the more powerful model 

## 🗂️ Integrating with BERTopic

You will remember that different aspects of the BERTopic process can be customised. Rather than running the embedding and representation steps locally, we will use the more powerful remote models to perform these steps. This should result in better, more accurate topics being generated, and offers a convenient way to quickly label them.

In [ ]:
from bertopic import BERTopic
from bertopic.backend import OpenAIBackend
from bertopic import representation
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('data/guardian_articles.csv')

# Filter the DataFrame for Opinion articles and drop rows with NaN in 'bodyContent'
sample_df = df[df['sectionName'] == 'Opinion'
               ].dropna(subset='bodyContent').sample(n=1000, random_state=42) # Sample 1000 rows for speed today
docs = sample_df['bodyContent'].tolist()

The BERTopic process will be tweaked as follows:

1. **Embedding**: Convert text documents into vector representations using a pre-trained model (like BERT). *This is now done remotely using OpenAI's embedding models.*
2. **Dimensionality Reduction**: Reduce the high-dimensional embeddings to a lower-dimensional space using techniques like UMAP. *Still run locally.*
3. **Clustering**: Group the reduced embeddings into clusters (i.e., topics) using algorithms like HDBSCAN. *Still run locally.*
4. **Tokenization**: Tokenize the clusters to prepare for topic representation. *Still run locally.*
5. **Weighting**: Calculate the importance of words in each cluster using class-based TF-IDF (c-TF-IDF) or similar methods. *Still run locally.*
6. **Topic Representation**: Provide a representation (and/or label) of each topic, based on the most important words in each cluster and, optionaly, example documents. *This is now done remotely using OpenAI's language models.*

The topic representation step queries the API with the following prompt:

```text
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the following format:
topic: <topic label>
```

and labels the topics accordingly

In [ ]:
embedding_model = OpenAIBackend(client, "text-embedding-3-small", batch_size=32) # Use embedding model from OpenAI, batch size prevents API limits being hit, adjust as necessary
representation_model = representation.OpenAI(client, model='gpt-4o-mini') # Use OpenAI model for topic representation, can be any OpenAI model that supports text generation
vectorizer_model = CountVectorizer(stop_words="english") # only used for c-TF-IDF stage

topic_model = BERTopic(embedding_model=embedding_model, # create BERTopic model with OpenAI embeddings
                       vectorizer_model=vectorizer_model, # used for c-TF-IDF as before
                       representation_model=representation_model) # used for topic representation

topics, probs = topic_model.fit_transform(docs) # fit the model to the documents
topic_info = topic_model.get_topic_info() # get topic information
topic_info # display the topics found

In [ ]:
topic_model.visualize_topics()

## ⚖️ Practicalities, ethics, and limitations
When using the advanced models, there are several practicalities, ethical considerations, and limitations to keep in mind:

- **Cost**: Using remote models can incur costs, especially for large datasets or complex tasks. It's important to monitor usage and budget accordingly.
- **Data Privacy**: When sending data to remote models, consider the sensitivity of the data. Ensure that you comply with data protection regulations, institutional ethical review, and that the service provider has appropriate security measures in place.
- **Bias and Fairness**: Machine learning models are know to exhibit various biases. It's crucial to firstly consider whether applying the model is appropriate for your task, and secondly to evaluate the model's performance across different demographic groups and ensure fairness in predictions.
- **Transparency, interpretability, and reproducibility**: These large models can be black boxes, making it difficult to understand how they arrive at their predictions. Moreover, the models are constantly being updated and improved, which can lead to inconsistencies in results over time. Finally, the models are not open source, so it is not possible to reproduce the results exactly as they were obtained. I would encourage the use of open source / open weight models.
- **Local models**: We have focused on remote models today, but there are many open source models available that can be run locally. These models can be more cost-effective and allow for greater control over the data and the model itself. However, they may require more computational resources and expertise to set up and use effectively. If this topic is of interest I would strongly encourage exploring [ollama](https://ollama.com/) and the [Hugging Face Hub](https://huggingface.co/), which has a wide range of open source models available for use, as well as dedicated coding resources to help with setup.


### 🏋️ Exercise (Free code / office hours)
1. Take anything you have learnt today, and try to apply it towards something from your own work. Feel free to discuss with me best approaches, or ask for help with the code.

## 🌯 Wrap up

We learnt how to query the OpenAI API and used it to perform text classification tasks - importantly making sure to validate our results. We also saw how to integrate the API into the BERTopic process to improve topical clusters and representation. Finaly, we reflected on essential practicalities, ethics, and limitations of NLP approaches.

If the topic is of interest to you going forward, I’d encourage you to learn more about the foundations of neural networks and deep learning for NLP. We’ve jumped straight to using remote LLM APIs as a convenient tool, but under the hood are models like transformers, trained on huge corpora using powerful architectures. Understanding these models from the ground up will help you build more interpretable, reliable, and responsible NLP systems.
